In [1]:
library("lme4")
library('afex')
library("dplyr")
library('emmeans')
set_sum_contrasts()

require(parallel)
(nc <- detectCores()) # number of cores
cl <- makeCluster(rep("localhost", nc))

Loading required package: Matrix

************
Welcome to afex. For support visit: http://afex.singmann.science/

- Functions for ANOVAs: aov_car(), aov_ez(), and aov_4()
- Methods for calculating p-values with mixed(): 'S', 'KR', 'LRT', and 'PB'
- 'afex_aov' and 'mixed' objects can be passed to emmeans() for follow-up tests
- Get and set global package options with: afex_options()
- Set sum-to-zero contrasts globally: set_sum_contrasts()
- For example analyses see: browseVignettes("afex")
************


Attaching package: 'afex'


The following object is masked from 'package:lme4':

    lmer



Attaching package: 'dplyr'


The following objects are masked from 'package:stats':

    filter, lag


The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union


setting contr.sum globally: options(contrasts=c('contr.sum', 'contr.poly'))

Loading required package: parallel



[1] 10

# figure 1

In [2]:
df_pr <- read.csv(file = '../data/rep/R/df_pr_for_R.csv', stringsAsFactors = TRUE)

str(df_pr)

df_p0r0 <- dplyr::filter(df_pr, cond_amount=='p0/r0')
df_cpr <- dplyr::filter(df_pr, cond_amount=='p/r')

'data.frame':	878 obs. of  17 variables:
 $ sub                 : Factor w/ 37 levels "0522-1","0522-2",..: 1 1 3 3 5 5 7 7 9 9 ...
 $ story               : int  3 3 3 3 3 3 3 3 3 3 ...
 $ segment             : Factor w/ 13 levels "3_01","3_02",..: 1 1 1 1 1 1 1 1 1 1 ...
 $ segment_count       : num  12 12 12 12 12 12 12 12 12 12 ...
 $ segment_num         : int  1 1 1 1 1 1 1 1 1 1 ...
 $ base_segment        : Factor w/ 13 levels "3_01","3_02",..: 2 2 2 2 2 2 2 2 2 2 ...
 $ segment_pair        : Factor w/ 12 levels "3_01-02","3_02-03",..: 1 1 1 1 1 1 1 1 1 1 ...
 $ condition           : Factor w/ 4 levels "p","p0","r","r0": 4 3 4 3 4 3 4 3 4 3 ...
 $ cond_direction      : Factor w/ 2 levels "b","f": 1 1 1 1 1 1 1 1 1 1 ...
 $ cond_amount         : Factor w/ 2 levels "p/r","p0/r0": 2 1 2 1 2 1 2 1 2 1 ...
 $ target_points       : num  15 15 15 15 15 15 15 15 15 15 ...
 $ e_total             : num  1 4 1 4.5 1 2.5 2 2 1 2.5 ...
 $ e_target            : num  0 4 0 3.5 0 2.5 1 1 0 1.5 ..

In [35]:
# pr_hit_ac proportion
pr_hit_ac <- mixed(cbind(e_target, target_points - e_target) ~ 
                   cond_direction*cond_amount*segment_count_demean + 
                  (cond_direction*cond_amount|segment) + 
                  (cond_amount*segment_count_demean|sub), 
                df_pr, family = binomial, method = "LRT", cl=cl)
pr_hit_ac
saveRDS(pr_hit_ac, "../stats_models/rep/pr_hit_ac.rds")

Numerical variables NOT centered on 0: segment_count_demean
If in interactions, interpretation of lower order (e.g., main) effects difficult.

Warning message:
"lme4 reported (at least) the following warnings for 'full':
  * failure to converge in 10000 evaluations
  * Model failed to converge with max|grad| = 1.41794 (tol = 0.002, component 1)"
Warning message:
"lme4 reported (at least) the following warnings for 'cond_direction':
  * failure to converge in 10000 evaluations
  * Model failed to converge with max|grad| = 0.843627 (tol = 0.002, component 1)"
Warning message:
"lme4 reported (at least) the following warnings for 'cond_amount':
  * failure to converge in 10000 evaluations
  * unable to evaluate scaled gradient
  * Model failed to converge: degenerate  Hessian with 1 negative eigenvalues"
Warning message:
"lme4 reported (at least) the following warnings for 'segment_count_demean':
  * failure to converge in 10000 evaluations
  * Model failed to converge with max|grad| = 0.7

Mixed Model Anova Table (Type 3 tests, LRT-method)

Model: cbind(e_target, target_points - e_target) ~ cond_direction * 
Model:     cond_amount * segment_count_demean + (cond_direction * cond_amount | 
Model:     segment) + (cond_amount * segment_count_demean | sub)
Data: df_pr
Df full model: 28
                                           Effect df     Chisq p.value
1                                  cond_direction  1      1.94    .163
2                                     cond_amount  1 27.42 ***   <.001
3                            segment_count_demean  1    6.42 *    .011
4                      cond_direction:cond_amount  1   8.35 **    .004
5             cond_direction:segment_count_demean  1    4.73 *    .030
6                cond_amount:segment_count_demean  1      0.29    .589
7 cond_direction:cond_amount:segment_count_demean  1   8.68 **    .003
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '+' 0.1 ' ' 1

In [36]:
p0r0_hit_ac <- mixed(cbind(e_target, target_points - e_target) ~ 
                   cond_direction*segment_count_demean + 
                  (cond_direction|segment) + 
                  (segment_count_demean|sub), 
                df_p0r0, family = binomial, method = "LRT", cl=cl)
p0r0_hit_ac
saveRDS(p0r0_hit_ac, "../stats_models/rep/p0r0_hit_ac.rds")

Numerical variables NOT centered on 0: segment_count_demean
If in interactions, interpretation of lower order (e.g., main) effects difficult.

Warning message:
"lme4 reported (at least) the following warnings for 'full':
  * Model failed to converge with max|grad| = 0.0263624 (tol = 0.002, component 1)"
Warning message:
"lme4 reported (at least) the following warnings for 'cond_direction':
  * boundary (singular) fit: see help('isSingular')"
Warning message:
"lme4 reported (at least) the following warnings for 'segment_count_demean':
  * Model failed to converge with max|grad| = 0.150668 (tol = 0.002, component 1)"
Warning message:
"lme4 reported (at least) the following warnings for 'cond_direction:segment_count_demean':
  * Model failed to converge with max|grad| = 0.0709186 (tol = 0.002, component 1)"


Mixed Model Anova Table (Type 3 tests, LRT-method)

Model: cbind(e_target, target_points - e_target) ~ cond_direction * 
Model:     segment_count_demean + (cond_direction | segment) + (segment_count_demean | 
Model:     sub)
Data: df_p0r0
Df full model: 10
                               Effect df     Chisq p.value
1                      cond_direction  1    5.73 *    .017
2                segment_count_demean  1    3.25 +    .072
3 cond_direction:segment_count_demean  1 11.42 ***   <.001
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '+' 0.1 ' ' 1

In [37]:
cpr_hit_ac <- mixed(cbind(e_target, target_points - e_target) ~ 
                   cond_direction*segment_count_demean + 
                  (cond_direction|segment) + 
                  (segment_count_demean|sub), 
                df_cpr, family = binomial, method = "LRT", cl=cl)
cpr_hit_ac
saveRDS(cpr_hit_ac, "../stats_models/rep/cpr_hit_ac.rds")

Numerical variables NOT centered on 0: segment_count_demean
If in interactions, interpretation of lower order (e.g., main) effects difficult.

Warning message:
"lme4 reported (at least) the following warnings for 'full':
  * Model failed to converge with max|grad| = 0.294395 (tol = 0.002, component 1)"
Warning message:
"lme4 reported (at least) the following warnings for 'cond_direction':
  * Model failed to converge with max|grad| = 0.308889 (tol = 0.002, component 1)"
Warning message:
"lme4 reported (at least) the following warnings for 'segment_count_demean':
  * Model failed to converge with max|grad| = 0.00481637 (tol = 0.002, component 1)"
Warning message:
"lme4 reported (at least) the following warnings for 'cond_direction:segment_count_demean':
  * Model failed to converge with max|grad| = 0.011799 (tol = 0.002, component 1)"


Mixed Model Anova Table (Type 3 tests, LRT-method)

Model: cbind(e_target, target_points - e_target) ~ cond_direction * 
Model:     segment_count_demean + (cond_direction | segment) + (segment_count_demean | 
Model:     sub)
Data: df_cpr
Df full model: 10
                               Effect df  Chisq p.value
1                      cond_direction  1   0.32    .569
2                segment_count_demean  1 4.82 *    .028
3 cond_direction:segment_count_demean  1   0.39    .532
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '+' 0.1 ' ' 1

In [38]:
# pr_simi_info
pr_simi_info <- mixed(res_1_simi_info_z ~ cond_direction*cond_amount*segment_count_demean + 
                  (cond_direction*cond_amount|segment) + 
                  (cond_amount*segment_count_demean|sub), 
                  df_pr, method='S')
pr_simi_info
saveRDS(pr_simi_info, "../stats_models/rep/pr_simi_info.rds")

Numerical variables NOT centered on 0: segment_count_demean
If in interactions, interpretation of lower order (e.g., main) effects difficult.

boundary (singular) fit: see help('isSingular')

Warning message:
"lme4 reported (at least) the following warnings for 'full':
  * boundary (singular) fit: see help('isSingular')"


Mixed Model Anova Table (Type 3 tests, S-method)

Model: res_1_simi_info_z ~ cond_direction * cond_amount * segment_count_demean + 
Model:     (cond_direction * cond_amount | segment) + (cond_amount * 
Model:     segment_count_demean | sub)
Data: df_pr
                                           Effect       df         F p.value
1                                  cond_direction 1, 25.03    5.72 *    .025
2                                     cond_amount 1, 12.13 98.21 ***   <.001
3                            segment_count_demean 1, 13.27      0.23    .642
4                      cond_direction:cond_amount 1, 16.51      0.27    .612
5             cond_direction:segment_count_demean 1, 11.73      0.19    .667
6                cond_amount:segment_count_demean 1, 15.92      1.88    .189
7 cond_direction:cond_amount:segment_count_demean 1, 12.39    3.94 +    .070
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '+' 0.1 ' ' 1

In [39]:
p0r0_simi_info <- mixed(res_1_simi_info_z ~ cond_direction*segment_count_demean + 
                  (cond_direction|segment) + 
                  (segment_count_demean|sub), 
                  df_p0r0, method='S')
p0r0_simi_info
saveRDS(p0r0_simi_info, "../stats_models/rep/p0r0_simi_info.rds")

Numerical variables NOT centered on 0: segment_count_demean
If in interactions, interpretation of lower order (e.g., main) effects difficult.

boundary (singular) fit: see help('isSingular')

Warning message:
"lme4 reported (at least) the following warnings for 'full':
  * boundary (singular) fit: see help('isSingular')"


Mixed Model Anova Table (Type 3 tests, S-method)

Model: res_1_simi_info_z ~ cond_direction * segment_count_demean + (cond_direction | 
Model:     segment) + (segment_count_demean | sub)
Data: df_p0r0
                               Effect       df      F p.value
1                      cond_direction 1, 16.85 3.46 +    .081
2                segment_count_demean 1, 11.82   0.02    .904
3 cond_direction:segment_count_demean 1, 12.80   0.41    .534
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '+' 0.1 ' ' 1

In [41]:
cpr_simi_info <- mixed(res_1_simi_info_z ~ cond_direction*segment_count_demean + 
                  (cond_direction|segment) + 
                  (segment_count_demean|sub), 
                  df_cpr, method='S')
cpr_simi_info
saveRDS(cpr_simi_info, "../stats_models/rep/cpr_simi_info.rds")

Numerical variables NOT centered on 0: segment_count_demean
If in interactions, interpretation of lower order (e.g., main) effects difficult.

boundary (singular) fit: see help('isSingular')

Warning message:
"lme4 reported (at least) the following warnings for 'full':
  * boundary (singular) fit: see help('isSingular')"


Mixed Model Anova Table (Type 3 tests, S-method)

Model: res_1_simi_info_z ~ cond_direction * segment_count_demean + (cond_direction | 
Model:     segment) + (segment_count_demean | sub)
Data: df_cpr
                               Effect       df      F p.value
1                      cond_direction 1, 25.09 5.22 *    .031
2                segment_count_demean  1, 9.94 3.35 +    .097
3 cond_direction:segment_count_demean 1, 11.43   1.52    .242
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '+' 0.1 ' ' 1

In [42]:
# pr_MD
pr_MD <- mixed(res_1_MD_sub_z ~ cond_direction*cond_amount*segment_count_demean + 
                     (cond_direction*cond_amount|segment) + 
                     (cond_amount*segment_count_demean|sub), 
                     df_pr, method='S')
pr_MD
saveRDS(pr_MD, "../stats_models/rep/pr_MD.rds")

Numerical variables NOT centered on 0: segment_count_demean
If in interactions, interpretation of lower order (e.g., main) effects difficult.

boundary (singular) fit: see help('isSingular')

Warning message:
"lme4 reported (at least) the following warnings for 'full':
  * boundary (singular) fit: see help('isSingular')"


Mixed Model Anova Table (Type 3 tests, S-method)

Model: res_1_MD_sub_z ~ cond_direction * cond_amount * segment_count_demean + 
Model:     (cond_direction * cond_amount | segment) + (cond_amount * 
Model:     segment_count_demean | sub)
Data: df_pr
                                           Effect       df         F p.value
1                                  cond_direction 1, 11.56      2.53    .139
2                                     cond_amount  1, 8.77 81.08 ***   <.001
3                            segment_count_demean 1, 11.23      0.00    .990
4                      cond_direction:cond_amount  1, 8.87      0.13    .723
5             cond_direction:segment_count_demean 1, 10.67      1.94    .192
6                cond_amount:segment_count_demean  1, 9.52      1.75    .217
7 cond_direction:cond_amount:segment_count_demean  1, 9.32      0.16    .697
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '+' 0.1 ' ' 1

In [43]:
p0r0_MD <- mixed(res_1_MD_sub_z ~ cond_direction*segment_count_demean + 
                  (cond_direction|segment) + 
                  (segment_count_demean|sub), 
                  df_p0r0, method='S')
p0r0_MD
saveRDS(p0r0_MD, "../stats_models/rep/p0r0_MD.rds")


Numerical variables NOT centered on 0: segment_count_demean
If in interactions, interpretation of lower order (e.g., main) effects difficult.



Mixed Model Anova Table (Type 3 tests, S-method)

Model: res_1_MD_sub_z ~ cond_direction * segment_count_demean + (cond_direction | 
Model:     segment) + (segment_count_demean | sub)
Data: df_p0r0
                               Effect       df    F p.value
1                      cond_direction 1, 11.38 0.31    .590
2                segment_count_demean 1, 11.46 1.05    .327
3 cond_direction:segment_count_demean 1, 10.95 1.83    .203
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '+' 0.1 ' ' 1

In [44]:

cpr_MD <- mixed(res_1_MD_sub_z ~ cond_direction*segment_count_demean + 
                  (cond_direction|segment) + 
                  (segment_count_demean|sub), 
                  df_cpr, method='S')
cpr_MD
saveRDS(cpr_MD, "../stats_models/rep/cpr_MD.rds")

Numerical variables NOT centered on 0: segment_count_demean
If in interactions, interpretation of lower order (e.g., main) effects difficult.

boundary (singular) fit: see help('isSingular')

Warning message:
"lme4 reported (at least) the following warnings for 'full':
  * boundary (singular) fit: see help('isSingular')"


Mixed Model Anova Table (Type 3 tests, S-method)

Model: res_1_MD_sub_z ~ cond_direction * segment_count_demean + (cond_direction | 
Model:     segment) + (segment_count_demean | sub)
Data: df_cpr
                               Effect       df      F p.value
1                      cond_direction 1, 12.49 3.21 +    .098
2                segment_count_demean  1, 9.87   2.33    .158
3 cond_direction:segment_count_demean 1, 10.67   1.11    .316
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '+' 0.1 ' ' 1

# figure 2

In [47]:
# p0r0_hit_econd_total poisson
df_p0r0$e_total_2 <- df_p0r0$e_total*2 # integers required

p0r0_hit_econd_total <- mixed(e_total_2 ~ cond_direction*segment_count_demean + 
                      (cond_direction|segment) + 
                      (segment_count_demean|sub), 
                    df_p0r0, family = poisson, method = "LRT", cl=cl)
p0r0_hit_econd_total
saveRDS(p0r0_hit_econd_total, "../stats_models/rep/p0r0_hit_econd_total.rds")

Numerical variables NOT centered on 0: segment_count_demean
If in interactions, interpretation of lower order (e.g., main) effects difficult.

Warning message:
"lme4 reported (at least) the following warnings for 'full':
  * boundary (singular) fit: see help('isSingular')"
Warning message:
"lme4 reported (at least) the following warnings for 'cond_direction':
  * boundary (singular) fit: see help('isSingular')"
Warning message:
"lme4 reported (at least) the following warnings for 'segment_count_demean':
  * Model failed to converge with max|grad| = 0.0123628 (tol = 0.002, component 1)"
Warning message:
"lme4 reported (at least) the following warnings for 'cond_direction:segment_count_demean':
  * boundary (singular) fit: see help('isSingular')"


Mixed Model Anova Table (Type 3 tests, LRT-method)

Model: e_total_2 ~ cond_direction * segment_count_demean + (cond_direction | 
Model:     segment) + (segment_count_demean | sub)
Data: df_p0r0
Df full model: 10
                               Effect df   Chisq p.value
1                      cond_direction  1    0.45    .502
2                segment_count_demean  1 9.74 **    .002
3 cond_direction:segment_count_demean  1    2.21    .137
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '+' 0.1 ' ' 1

In [4]:
df_pr_econd_bin <- read.csv(file = '../data/rep/R/df_pr_econd_bin_for_R.csv', stringsAsFactors = TRUE)
df_pr_econd_bin$n_hits_2 <- df_pr_econd_bin$n_hits*2 # integers required
str(df_pr_econd_bin)

df_p0r0_econd_bin <- dplyr::filter(df_pr_econd_bin, cond_amount=='p0/r0')
df_cpr_econd_bin <- dplyr::filter(df_pr_econd_bin, cond_amount=='p/r')

'data.frame':	1756 obs. of  15 variables:
 $ sub                 : Factor w/ 37 levels "0522-1","0522-2",..: 1 1 3 3 5 5 7 7 9 9 ...
 $ story               : int  3 3 3 3 3 3 3 3 3 3 ...
 $ segment             : Factor w/ 13 levels "3_01","3_02",..: 1 1 1 1 1 1 1 1 1 1 ...
 $ segment_count       : num  12 12 12 12 12 12 12 12 12 12 ...
 $ segment_num         : int  1 1 1 1 1 1 1 1 1 1 ...
 $ base_segment        : Factor w/ 13 levels "3_01","3_02",..: 2 2 2 2 2 2 2 2 2 2 ...
 $ segment_count_demean: num  5.5 5.5 5.5 5.5 5.5 5.5 5.5 5.5 5.5 5.5 ...
 $ segment_num_demean  : int  -6 -6 -6 -6 -6 -6 -6 -6 -6 -6 ...
 $ segment_pair        : Factor w/ 12 levels "3_01-02","3_02-03",..: 1 1 1 1 1 1 1 1 1 1 ...
 $ condition           : Factor w/ 4 levels "p","p0","r","r0": 4 3 4 3 4 3 4 3 4 3 ...
 $ cond_direction      : Factor w/ 2 levels "b","f": 1 1 1 1 1 1 1 1 1 1 ...
 $ cond_amount         : Factor w/ 2 levels "p/r","p0/r0": 2 1 2 1 2 1 2 1 2 1 ...
 $ e_cond_bin          : Factor w/ 2 levels

In [ ]:
p0r0_hit_econd_bin <- mixed(n_hits_2 ~ 
                   cond_direction*segment_count_demean*e_cond_bin + 
                  (cond_direction*e_cond_bin|segment) + 
                  (segment_count_demean*e_cond_bin|sub), 
                df_p0r0_econd_bin, family = poisson, method = "LRT", cl=cl)
p0r0_hit_econd_bin
saveRDS(p0r0_hit_econd_bin, "../stats_models/rep/p0r0_hit_econd_bin.rds")

In [3]:
df_pr_econd <- read.csv(file = '../data/rep/R/df_pr_econd_for_R.csv', stringsAsFactors = TRUE)
df_pr_econd$n_hits_2 <- df_pr_econd$n_hits*2 # integers required
str(df_pr_econd)

df_p0r0_econd <- dplyr::filter(df_pr_econd, cond_amount=='p0/r0')
df_cpr_econd <- dplyr::filter(df_pr_econd, cond_amount=='p/r')

'data.frame':	3512 obs. of  15 variables:
 $ sub                 : Factor w/ 37 levels "0522-1","0522-2",..: 1 1 3 3 5 5 7 7 9 9 ...
 $ story               : int  3 3 3 3 3 3 3 3 3 3 ...
 $ segment             : Factor w/ 13 levels "3_01","3_02",..: 1 1 1 1 1 1 1 1 1 1 ...
 $ segment_count       : num  12 12 12 12 12 12 12 12 12 12 ...
 $ segment_num         : int  1 1 1 1 1 1 1 1 1 1 ...
 $ base_segment        : Factor w/ 13 levels "3_01","3_02",..: 2 2 2 2 2 2 2 2 2 2 ...
 $ segment_count_demean: num  5.5 5.5 5.5 5.5 5.5 5.5 5.5 5.5 5.5 5.5 ...
 $ segment_num_demean  : int  -6 -6 -6 -6 -6 -6 -6 -6 -6 -6 ...
 $ segment_pair        : Factor w/ 12 levels "3_01-02","3_02-03",..: 1 1 1 1 1 1 1 1 1 1 ...
 $ condition           : Factor w/ 4 levels "p","p0","r","r0": 4 3 4 3 4 3 4 3 4 3 ...
 $ cond_direction      : Factor w/ 2 levels "b","f": 1 1 1 1 1 1 1 1 1 1 ...
 $ cond_amount         : Factor w/ 2 levels "p/r","p0/r0": 2 1 2 1 2 1 2 1 2 1 ...
 $ e_cond              : Factor w/ 4 levels

In [49]:
p0r0_hit_econd <- mixed(n_hits_2 ~ 
                   cond_direction*segment_count_demean*e_cond + 
                  (cond_direction*e_cond|segment) + 
                  (segment_count_demean*e_cond|sub), 
                df_p0r0_econd, family = poisson, method = "LRT", cl=cl)
p0r0_hit_econd
saveRDS(p0r0_hit_econd, "../stats_models/rep/p0r0_hit_econd.rds")

Numerical variables NOT centered on 0: segment_count_demean
If in interactions, interpretation of lower order (e.g., main) effects difficult.

Warning message in mixed(n_hits_2 ~ cond_direction * segment_count_demean * e_cond + :
"Following nested model(s) provide better fit than full model: segment_count_demean, segment_count_demean:e_cond 
  Results cannot be trusted. Try all_fit=TRUE or reduce random effect structure!"
Warning message:
"Following nested model(s) provide better fit than full model: segment_count_demean, segment_count_demean:e_cond 
  Results cannot be trusted. Try all_fit=TRUE!"
Warning message:
"lme4 reported (at least) the following warnings for 'full':
  * failure to converge in 10000 evaluations
  * unable to evaluate scaled gradient
  * Model failed to converge: degenerate  Hessian with 6 negative eigenvalues"
Warning message:
"lme4 reported (at least) the following warnings for 'cond_direction':
  * failure to converge in 10000 evaluations
  * unable to evaluat

Mixed Model Anova Table (Type 3 tests, LRT-method)

Model: n_hits_2 ~ cond_direction * segment_count_demean * e_cond + (cond_direction * 
Model:     e_cond | segment) + (segment_count_demean * e_cond | sub)
Data: df_p0r0_econd
Df full model: 88
                                      Effect df     Chisq p.value
1                             cond_direction  1    5.42 *    .020
2                       segment_count_demean  1      0.00   >.999
3                                     e_cond  3 68.88 ***   <.001
4        cond_direction:segment_count_demean  1 12.58 ***   <.001
5                      cond_direction:e_cond  3 25.10 ***   <.001
6                segment_count_demean:e_cond  3      0.00   >.999
7 cond_direction:segment_count_demean:e_cond  3  14.65 **    .002
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '+' 0.1 ' ' 1

## step plot

### original

In [6]:
df_refer_long_p0r0 <- read.csv(file = '../data/rep/R/df_refer_long_p0r0_for_R.csv', stringsAsFactors = TRUE)
df_refer_long_p0r0$refer_seg_e_2 <- df_refer_long_p0r0$refer_seg_e*2 # integers required
str(df_refer_long_p0r0)

'data.frame':	5982 obs. of  15 variables:
 $ condition           : Factor w/ 2 levels "p0","r0": 2 2 2 2 2 2 2 2 2 2 ...
 $ story               : int  3 3 3 3 3 3 3 3 3 3 ...
 $ sub                 : Factor w/ 37 levels "0522-1","0522-2",..: 1 3 5 7 9 11 13 15 17 19 ...
 $ segment_count       : num  12 12 12 12 12 12 12 12 12 12 ...
 $ segment_count_demean: num  5.5 5.5 5.5 5.5 5.5 5.5 5.5 5.5 5.5 5.5 ...
 $ main_or_gap         : Factor w/ 2 levels "gap","main": 1 1 1 1 1 1 1 1 1 1 ...
 $ base_segment        : Factor w/ 13 levels "3_01","3_02",..: 2 2 2 2 2 2 2 2 2 2 ...
 $ step_abs            : num  1.5 1.5 1.5 1.5 1.5 1.5 1.5 1.5 1.5 1.5 ...
 $ step_abs_demean     : num  -4.12 -4.12 -4.12 -4.12 -4.12 ...
 $ step_abs_demean_old : num  -3.12 -3.12 -3.12 -3.12 -3.12 ...
 $ refer_seg           : Factor w/ 26 levels "3_0.5","3_1.0",..: 1 1 1 1 1 1 1 1 1 1 ...
 $ base_refer_seg_pair : Factor w/ 324 levels "3_01_3_1.5","3_01_3_10.0",..: 25 25 25 25 25 25 25 25 25 25 ...
 $ refer_seg_e      

In [3]:
p0_r0_hit_refer_step_2 <- mixed(refer_seg_e_2 ~ condition*main_or_gap*step_abs_demean + 
                           (condition|base_segment) + 
                           (1|base_refer_seg_pair) +
                           (main_or_gap*step_abs_demean|sub), 
                       df_refer_long_p0r0, family = poisson, method = "LRT", cl=cl)
p0_r0_hit_refer_step_2
saveRDS(p0_r0_hit_refer_step_2, "../stats_models/rep/p0_r0_hit_refer_step_2.rds")

Numerical variables NOT centered on 0: step_abs_demean
If in interactions, interpretation of lower order (e.g., main) effects difficult.

Warning message:
"lme4 reported (at least) the following warnings for 'full':
  * failure to converge in 10000 evaluations
  * unable to evaluate scaled gradient
  * Model failed to converge: degenerate  Hessian with 2 negative eigenvalues"
Warning message:
"lme4 reported (at least) the following warnings for 'condition':
  * failure to converge in 10000 evaluations
  * unable to evaluate scaled gradient
  * Model failed to converge: degenerate  Hessian with 1 negative eigenvalues"
Warning message:
"lme4 reported (at least) the following warnings for 'main_or_gap':
  * failure to converge in 10000 evaluations
  * Model failed to converge with max|grad| = 0.585693 (tol = 0.002, component 1)"
Warning message:
"lme4 reported (at least) the following warnings for 'step_abs_demean':
  * failure to converge in 10000 evaluations
  * Model failed to converge

Mixed Model Anova Table (Type 3 tests, LRT-method)

Model: refer_seg_e_2 ~ condition * main_or_gap * step_abs_demean + (condition | 
Model:     base_segment) + (1 | base_refer_seg_pair) + (main_or_gap * 
Model:     step_abs_demean | sub)
Data: df_refer_long_p0r0
Df full model: 22
                                 Effect df     Chisq p.value
1                             condition  1 23.45 ***   <.001
2                           main_or_gap  1   7.49 **    .006
3                       step_abs_demean  1   8.76 **    .003
4                 condition:main_or_gap  1    5.21 *    .023
5             condition:step_abs_demean  1    5.58 *    .018
6           main_or_gap:step_abs_demean  1      1.95    .163
7 condition:main_or_gap:step_abs_demean  1      0.17    .679
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '+' 0.1 ' ' 1

# figure 3

In [7]:
df_seg_seg_type_p0r0 <- read.csv(file = '../data/rep/R/df_seg_seg_type_p0r0_for_R.csv', stringsAsFactors = TRUE)
df_seg_seg_type_p0r0$story <- factor(df_seg_seg_type_p0r0$story)
str(df_seg_seg_type_p0r0)

'data.frame':	6223 obs. of  19 variables:
 $ sub                     : Factor w/ 37 levels "0522-1","0522-2",..: 1 1 1 1 1 1 1 1 1 1 ...
 $ condition               : Factor w/ 2 levels "p0","r0": 2 2 2 2 2 2 2 2 2 2 ...
 $ story                   : Factor w/ 1 level "3": 1 1 1 1 1 1 1 1 1 1 ...
 $ segment                 : Factor w/ 13 levels "3_01","3_02",..: 1 1 1 1 1 2 2 2 2 2 ...
 $ segment_count           : num  12 12 12 12 12 11 11 11 11 11 ...
 $ base_seg_num            : int  2 2 2 2 2 3 3 3 3 3 ...
 $ base_segment            : Factor w/ 13 levels "3_01","3_02",..: 2 2 2 2 2 3 3 3 3 3 ...
 $ ref_seg_num             : num  0.5 0.5 1 1 1.5 0.5 0.5 1 1 1.5 ...
 $ ref_step_base           : num  -1.5 -1.5 -1 -1 -0.5 -2.5 -2.5 -2 -2 -1.5 ...
 $ ref_step_base_abs       : num  1.5 1.5 1 1 0.5 2.5 2.5 2 2 1.5 ...
 $ ref_seg_type            : Factor w/ 4 levels "ahead_gap","ahead_main",..: 1 1 4 4 3 1 1 2 2 1 ...
 $ main_or_gap             : Factor w/ 2 levels "gap","main": 1 1 2 2 1 1 1

In [11]:
p0_r0_seg_seg_type_1 <- mixed(cbind(total_hits, total_points - total_hits) ~ 
                              condition*refer_type_merge*main_or_gap + ref_step_base_abs_demean + 
                              (condition|base_segment) + 
                              (1|base_refer_seg_pair) +
                              (refer_type_merge*main_or_gap + ref_step_base_abs_demean|sub), 
                          df_seg_seg_type_p0r0, family = binomial, method = "LRT", cl=cl)
p0_r0_seg_seg_type_1
saveRDS(p0_r0_seg_seg_type_1, "../stats_models/rep/p0_r0_seg_seg_type_1.rds")

Numerical variables NOT centered on 0: ref_step_base_abs_demean
If in interactions, interpretation of lower order (e.g., main) effects difficult.

Warning message:
"lme4 reported (at least) the following warnings for 'full':
  * failure to converge in 10000 evaluations
  * Model failed to converge with max|grad| = 0.556195 (tol = 0.002, component 1)"
Warning message:
"lme4 reported (at least) the following warnings for 'condition':
  * failure to converge in 10000 evaluations
  * Model failed to converge with max|grad| = 0.68736 (tol = 0.002, component 1)"
Warning message:
"lme4 reported (at least) the following warnings for 'refer_type_merge':
  * failure to converge in 10000 evaluations
  * Model failed to converge with max|grad| = 0.478927 (tol = 0.002, component 1)"
Warning message:
"lme4 reported (at least) the following warnings for 'main_or_gap':
  * failure to converge in 10000 evaluations
  * Model failed to converge with max|grad| = 0.472882 (tol = 0.002, component 1)"
Warnin

Mixed Model Anova Table (Type 3 tests, LRT-method)

Model: cbind(total_hits, total_points - total_hits) ~ condition * refer_type_merge * 
Model:     main_or_gap + ref_step_base_abs_demean + (condition | base_segment) + 
Model:     (1 | base_refer_seg_pair) + (refer_type_merge * main_or_gap + 
Model:     ref_step_base_abs_demean | sub)
Data: df_seg_seg_type_p0r0
Df full model: 28
                                  Effect df     Chisq p.value
1                              condition  1  10.58 **    .001
2                       refer_type_merge  1      0.31    .576
3                            main_or_gap  1 11.34 ***   <.001
4               ref_step_base_abs_demean  1   8.42 **    .004
5             condition:refer_type_merge  1      1.48    .223
6                  condition:main_or_gap  1 18.30 ***   <.001
7           refer_type_merge:main_or_gap  1 12.01 ***   <.001
8 condition:refer_type_merge:main_or_gap  1    4.17 *    .041
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '+' 0.1

# figure 4

In [8]:
df_seg_seg_boost_type_p0r0 <- read.csv(file = '../data/rep/R/df_seg_seg_boost_type_p0r0_for_R.csv', stringsAsFactors = TRUE)
df_seg_seg_boost_type_p0r0$story <- factor(df_seg_seg_boost_type_p0r0$story)
str(df_seg_seg_boost_type_p0r0)

'data.frame':	6223 obs. of  20 variables:
 $ sub                     : Factor w/ 37 levels "0522-1","0522-2",..: 1 1 1 1 1 1 1 1 1 1 ...
 $ condition               : Factor w/ 2 levels "p0","r0": 2 2 2 2 2 2 2 2 2 2 ...
 $ story                   : Factor w/ 1 level "3": 1 1 1 1 1 1 1 1 1 1 ...
 $ segment                 : Factor w/ 13 levels "3_01","3_02",..: 1 1 1 1 1 2 2 2 2 2 ...
 $ segment_count           : num  12 12 12 12 12 11 11 11 11 11 ...
 $ base_segment            : Factor w/ 13 levels "3_01","3_02",..: 2 2 2 2 2 3 3 3 3 3 ...
 $ base_seg_num            : int  2 2 2 2 2 3 3 3 3 3 ...
 $ ref_seg_num             : num  0.5 0.5 1 1 1.5 0.5 0.5 1 1 1.5 ...
 $ ref_step_base           : num  -1.5 -1.5 -1 -1 -0.5 -2.5 -2.5 -2 -2 -1.5 ...
 $ ref_step_base_abs       : num  1.5 1.5 1 1 0.5 2.5 2.5 2 2 1.5 ...
 $ ref_seg_type            : Factor w/ 4 levels "ahead_gap","ahead_main",..: 1 1 4 4 3 1 1 2 2 1 ...
 $ main_or_gap             : Factor w/ 2 levels "gap","main": 1 1 2 2 1 1 1

In [18]:
p0_r0_seg_seg_boost_type_tri_1 <- mixed(cbind(total_hits, total_points - total_hits) ~ 
                              condition*refer_boost_type_tri*main_or_gap + ref_step_base_abs_demean + 
                              (condition|base_segment) + 
                              (1|base_refer_seg_pair) +
                              (refer_boost_type_tri*main_or_gap + ref_step_base_abs_demean|sub), 
                          df_seg_seg_boost_type_p0r0, family = binomial, method = "LRT", cl=cl)
p0_r0_seg_seg_boost_type_tri_1
saveRDS(p0_r0_seg_seg_boost_type_tri_1, "../stats_models/rep/p0_r0_seg_seg_boost_type_tri_1.rds")

Numerical variables NOT centered on 0: ref_step_base_abs_demean
If in interactions, interpretation of lower order (e.g., main) effects difficult.

Warning message:
"lme4 reported (at least) the following warnings for 'full':
  * failure to converge in 10000 evaluations
  * unable to evaluate scaled gradient
  * Model failed to converge: degenerate  Hessian with 2 negative eigenvalues"
Warning message:
"lme4 reported (at least) the following warnings for 'condition':
  * failure to converge in 10000 evaluations
  * Model failed to converge with max|grad| = 1.04105 (tol = 0.002, component 1)"
Warning message:
"lme4 reported (at least) the following warnings for 'refer_boost_type_tri':
  * failure to converge in 10000 evaluations
  * unable to evaluate scaled gradient
  * Model failed to converge: degenerate  Hessian with 2 negative eigenvalues"
Warning message:
"lme4 reported (at least) the following warnings for 'main_or_gap':
  * failure to converge in 10000 evaluations
  * unable to e

Mixed Model Anova Table (Type 3 tests, LRT-method)

Model: cbind(total_hits, total_points - total_hits) ~ condition * refer_boost_type_tri * 
Model:     main_or_gap + ref_step_base_abs_demean + (condition | base_segment) + 
Model:     (1 | base_refer_seg_pair) + (refer_boost_type_tri * main_or_gap + 
Model:     ref_step_base_abs_demean | sub)
Data: df_seg_seg_boost_type_p0r0
Df full model: 45
                                      Effect df     Chisq p.value
1                                  condition  1    6.42 *    .011
2                       refer_boost_type_tri  2      2.69    .260
3                                main_or_gap  1    3.84 +    .050
4                   ref_step_base_abs_demean  1  10.19 **    .001
5             condition:refer_boost_type_tri  2      1.84    .399
6                      condition:main_or_gap  1    3.09 +    .079
7           refer_boost_type_tri:main_or_gap  2 23.14 ***   <.001
8 condition:refer_boost_type_tri:main_or_gap  2    7.29 *    .026
---
Signif

Warning message in gzfile(file, mode):
"cannot open compressed file './stats_models/p0_r0_seg_seg_boost_type_tri_1.rds', probable reason 'No such file or directory'"


ERROR: Error in gzfile(file, mode): cannot open the connection


### storyline

In [9]:
df_seg_seg_boost_type_sl_p0r0_for_R <- read.csv(file = '../data/rep/R/df_seg_seg_boost_type_sl_p0r0_for_R.csv', stringsAsFactors = TRUE)
df_seg_seg_boost_type_sl_p0r0_for_R$story <- factor(df_seg_seg_boost_type_sl_p0r0_for_R$story)
str(df_seg_seg_boost_type_sl_p0r0_for_R)

'data.frame':	7434 obs. of  19 variables:
 $ sub                     : Factor w/ 37 levels "0522-1","0522-2",..: 1 1 1 1 1 1 1 1 1 1 ...
 $ condition               : Factor w/ 2 levels "p0","r0": 2 2 2 2 2 2 2 2 2 2 ...
 $ story                   : Factor w/ 1 level "3": 1 1 1 1 1 1 1 1 1 1 ...
 $ segment                 : Factor w/ 13 levels "3_01","3_02",..: 1 1 1 1 1 2 2 2 2 2 ...
 $ segment_count           : num  12 12 12 12 12 11 11 11 11 11 ...
 $ base_segment            : Factor w/ 13 levels "3_01","3_02",..: 2 2 2 2 2 3 3 3 3 3 ...
 $ base_seg_num            : int  2 2 2 2 2 3 3 3 3 3 ...
 $ ref_seg_num             : num  0.5 0.5 1 1 1.5 0.5 0.5 1 1 1.5 ...
 $ ref_step_base           : num  -1.5 -1.5 -1 -1 -0.5 -2.5 -2.5 -2 -2 -1.5 ...
 $ ref_step_base_abs       : num  1.5 1.5 1 1 0.5 2.5 2.5 2 2 1.5 ...
 $ ref_seg_type            : Factor w/ 4 levels "ahead_gap","ahead_main",..: 1 1 4 4 3 1 1 2 2 1 ...
 $ main_or_gap             : Factor w/ 2 levels "gap","main": 1 1 2 2 1 1 1

In [27]:
p0_r0_seg_seg_boost_type_sl_tri_1 <- mixed(cbind(total_hits, total_points - total_hits) ~ 
                              condition*refer_boost_type_sl*main_or_gap + ref_step_base_abs_demean + 
                              (condition|base_segment) + 
                              (1|base_refer_seg_pair) +
                              (refer_boost_type_sl*main_or_gap + ref_step_base_abs_demean|sub), 
                          df_seg_seg_boost_type_sl_p0r0_for_R, family = binomial, method = "LRT", cl=cl)
p0_r0_seg_seg_boost_type_sl_tri_1
saveRDS(p0_r0_seg_seg_boost_type_sl_tri_1, "../stats_models/rep/p0_r0_seg_seg_boost_type_sl_tri_1.rds")

Numerical variables NOT centered on 0: ref_step_base_abs_demean
If in interactions, interpretation of lower order (e.g., main) effects difficult.

Warning message:
"lme4 reported (at least) the following warnings for 'full':
  * failure to converge in 10000 evaluations
  * Model failed to converge with max|grad| = 0.350211 (tol = 0.002, component 1)
  * Model is nearly unidentifiable: large eigenvalue ratio
 - Rescale variables?"
Warning message:
"lme4 reported (at least) the following warnings for 'condition':
  * failure to converge in 10000 evaluations
  * Model failed to converge with max|grad| = 0.351364 (tol = 0.002, component 1)"
Warning message:
"lme4 reported (at least) the following warnings for 'refer_boost_type_sl':
  * failure to converge in 10000 evaluations
  * unable to evaluate scaled gradient
  * Model failed to converge: degenerate  Hessian with 1 negative eigenvalues"
Warning message:
"lme4 reported (at least) the following warnings for 'main_or_gap':
  * failure to

Mixed Model Anova Table (Type 3 tests, LRT-method)

Model: cbind(total_hits, total_points - total_hits) ~ condition * refer_boost_type_sl * 
Model:     main_or_gap + ref_step_base_abs_demean + (condition | base_segment) + 
Model:     (1 | base_refer_seg_pair) + (refer_boost_type_sl * main_or_gap + 
Model:     ref_step_base_abs_demean | sub)
Data: df_seg_seg_boost_type_sl_p0r0_for_R
Df full model: 45
                                     Effect df    Chisq p.value
1                                 condition  1   6.42 *    .011
2                       refer_boost_type_sl  2     2.44    .296
3                               main_or_gap  1   3.43 +    .064
4                  ref_step_base_abs_demean  1   3.32 +    .069
5             condition:refer_boost_type_sl  2     1.26    .534
6                     condition:main_or_gap  1   6.12 *    .013
7           refer_boost_type_sl:main_or_gap  2 11.91 **    .003
8 condition:refer_boost_type_sl:main_or_gap  2 10.72 **    .005
---
Signif. codes:  0

# figure 5

In [10]:
df_seg_seg_rr_type_p0r0_main <- read.csv(file = '../data/rep/R/df_seg_seg_rr_type_p0r0_main_for_R.csv', stringsAsFactors = TRUE)
df_seg_seg_rr_type_p0r0_main$story <- factor(df_seg_seg_rr_type_p0r0_main$story)
str(df_seg_seg_rr_type_p0r0_main)

'data.frame':	4073 obs. of  19 variables:
 $ sub                     : Factor w/ 37 levels "0522-1","0522-2",..: 1 1 1 1 1 1 1 1 1 1 ...
 $ condition               : Factor w/ 2 levels "p0","r0": 2 2 2 2 2 2 2 2 2 2 ...
 $ story                   : Factor w/ 1 level "3": 1 1 1 1 1 1 1 1 1 1 ...
 $ segment                 : Factor w/ 13 levels "3_01","3_02",..: 1 1 2 2 2 3 3 3 3 3 ...
 $ segment_count           : num  12 12 11 11 11 10 10 10 10 10 ...
 $ base_seg_num            : int  2 2 3 3 3 4 4 4 4 4 ...
 $ base_segment            : Factor w/ 13 levels "3_01","3_02",..: 2 2 3 3 3 4 4 4 4 4 ...
 $ ref_seg_num             : num  1 1 1 1 2 1 1 2 3 3 ...
 $ ref_step_base           : num  -1 -1 -2 -2 -1 -3 -3 -2 -1 -1 ...
 $ ref_step_base_abs       : num  1 1 2 2 1 3 3 2 1 1 ...
 $ ref_seg_type            : Factor w/ 2 levels "ahead_main","target": 2 2 1 1 2 1 1 1 2 2 ...
 $ main_or_gap             : Factor w/ 1 level "main": 1 1 1 1 1 1 1 1 1 1 ...
 $ rr_type                 : Factor w/

In [27]:
p0_r0_seg_seg_rr_type_1 <- mixed(cbind(total_hits, total_points - total_hits) ~ 
                               condition*rr_type + ref_step_base_abs_demean + 
                              (condition|base_segment) + 
                              (1|base_refer_seg_pair) +
                              (rr_type + ref_step_base_abs_demean|sub), 
                              df_seg_seg_rr_type_p0r0_main, family = binomial, method = "LRT", cl=cl)
p0_r0_seg_seg_rr_type_1
saveRDS(p0_r0_seg_seg_rr_type_1, "../stats_models/rep/p0_r0_seg_seg_rr_type_1.rds")

Numerical variables NOT centered on 0: ref_step_base_abs_demean
If in interactions, interpretation of lower order (e.g., main) effects difficult.

Warning message:
"lme4 reported (at least) the following warnings for 'full':
  * boundary (singular) fit: see help('isSingular')"
Warning message:
"lme4 reported (at least) the following warnings for 'condition':
  * failure to converge in 10000 evaluations
  * unable to evaluate scaled gradient
  * Model failed to converge: degenerate  Hessian with 2 negative eigenvalues"
Warning message:
"lme4 reported (at least) the following warnings for 'rr_type':
  * Model failed to converge with max|grad| = 0.0800653 (tol = 0.002, component 1)
  * Model is nearly unidentifiable: very large eigenvalue
 - Rescale variables?"
Warning message:
"lme4 reported (at least) the following warnings for 'ref_step_base_abs_demean':
  * boundary (singular) fit: see help('isSingular')"
Warning message:
"lme4 reported (at least) the following warnings for 'condition

Mixed Model Anova Table (Type 3 tests, LRT-method)

Model: cbind(total_hits, total_points - total_hits) ~ condition * rr_type + 
Model:     ref_step_base_abs_demean + (condition | base_segment) + (1 | 
Model:     base_refer_seg_pair) + (rr_type + ref_step_base_abs_demean | 
Model:     sub)
Data: df_seg_seg_rr_type_p0r0_main
Df full model: 21
                    Effect df     Chisq p.value
1                condition  1      0.07    .797
2                  rr_type  2 27.90 ***   <.001
3 ref_step_base_abs_demean  1 12.17 ***   <.001
4        condition:rr_type  2      0.42    .813
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '+' 0.1 ' ' 1

# social

In [11]:
df_seg_seg_social_p0r0_main <- read.csv(file = '../data/rep/R/df_seg_seg_social_p0r0_main_for_R.csv', stringsAsFactors = TRUE)
str(df_seg_seg_social_p0r0_main)

'data.frame':	3768 obs. of  19 variables:
 $ sub                     : Factor w/ 37 levels "0522-1","0522-2",..: 1 1 1 1 1 1 1 1 1 1 ...
 $ condition               : Factor w/ 2 levels "p0","r0": 2 2 2 2 2 2 2 2 2 2 ...
 $ story                   : int  3 3 3 3 3 3 3 3 3 3 ...
 $ segment                 : Factor w/ 13 levels "3_01","3_02",..: 1 1 2 2 2 3 3 3 3 4 ...
 $ segment_count           : num  12 12 11 11 11 10 10 10 10 9 ...
 $ base_seg_num            : int  2 2 3 3 3 4 4 4 4 5 ...
 $ base_segment            : Factor w/ 13 levels "3_01","3_02",..: 2 2 3 3 3 4 4 4 4 5 ...
 $ ref_seg_num             : num  1 1 1 1 2 1 1 2 3 1 ...
 $ ref_step_base           : num  -1 -1 -2 -2 -1 -3 -3 -2 -1 -4 ...
 $ ref_step_base_abs       : num  1 1 2 2 1 3 3 2 1 4 ...
 $ ref_seg_type            : Factor w/ 2 levels "ahead_main","target": 2 2 1 1 2 1 1 1 2 1 ...
 $ main_or_gap             : Factor w/ 1 level "main": 1 1 1 1 1 1 1 1 1 1 ...
 $ social.non.social       : Factor w/ 2 levels "n","s": 

In [40]:
p0_r0_seg_seg_social_1 <- mixed(cbind(total_hits, total_points - total_hits) ~ 
                               condition*social + ref_step_base_abs_demean + 
                              (condition|base_segment) + 
                              (1|base_refer_seg_pair) +
                              (social + ref_step_base_abs_demean|sub), 
                              df_seg_seg_social_p0r0_main, family = binomial, method = "LRT", cl=cl)
p0_r0_seg_seg_social_1
saveRDS(p0_r0_seg_seg_social_1, "../stats_models/rep/p0_r0_seg_seg_social_1.rds")

Numerical variables NOT centered on 0: ref_step_base_abs_demean
If in interactions, interpretation of lower order (e.g., main) effects difficult.

Warning message:
"lme4 reported (at least) the following warnings for 'full':
  * Model failed to converge with max|grad| = 0.386246 (tol = 0.002, component 1)
  * Model is nearly unidentifiable: large eigenvalue ratio
 - Rescale variables?"
Warning message:
"lme4 reported (at least) the following warnings for 'condition':
  * Model failed to converge with max|grad| = 0.121678 (tol = 0.002, component 1)"
Warning message:
"lme4 reported (at least) the following warnings for 'social':
  * Model failed to converge with max|grad| = 0.213942 (tol = 0.002, component 1)"
Warning message:
"lme4 reported (at least) the following warnings for 'ref_step_base_abs_demean':
  * Model failed to converge with max|grad| = 0.0879 (tol = 0.002, component 1)
  * Model is nearly unidentifiable: large eigenvalue ratio
 - Rescale variables?"
Warning message:
"lme4

Mixed Model Anova Table (Type 3 tests, LRT-method)

Model: cbind(total_hits, total_points - total_hits) ~ condition * social + 
Model:     ref_step_base_abs_demean + (condition | base_segment) + (1 | 
Model:     base_refer_seg_pair) + (social + ref_step_base_abs_demean | 
Model:     sub)
Data: df_seg_seg_social_p0r0_main
Df full model: 15
                    Effect df     Chisq p.value
1                condition  1    6.46 *    .011
2                   social  1    3.49 +    .062
3 ref_step_base_abs_demean  1 12.27 ***   <.001
4         condition:social  1    4.72 *    .030
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '+' 0.1 ' ' 1